## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image
from torchvision import datasets, models
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import time as time
import copy
from torchvision.datasets import OxfordIIITPet
from torch.optim import lr_scheduler

from tqdm.notebook import tqdm

## Fields

In [2]:



num_classes = 37 #Number of classes in dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image_size = 224
batch_size = 128

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

path = "Deep_Learning_Project-1/Data"

/home/da150x_47/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


## Methods

In [3]:
def set_parameter_requires_grad(model, feature_extracting):
    params_to_update = list(model.named_parameters())
    
    if feature_extracting:
        for name, param in params_to_update:
            param.requires_grad = False      
    else:
        for module, param in zip(model.modules(), model.parameters()):
            if isinstance(module, nn.BatchNorm2d):
                param.requires_grad = False
            else:
                param.requires_grad = True
            



def initialize_model(model_name, num_classes, feature_extracting): #Initialize Resnet

    if model_name == "resnet":

        model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True) 
        
        set_parameter_requires_grad(model, feature_extracting)
        
        num_ftrs = model.fc.in_features
        head = nn.Sequential(nn.BatchNorm1d(num_ftrs),
                             nn.ReLU(),
                             nn.Dropout(0.25),
                nn.Linear(num_ftrs, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),
                nn.Dropout(0.25),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(256, num_classes))
        
        model.fc = nn.Linear(num_ftrs,num_classes) # Update last layer to binary classification (Dog/cat)
        input_size = 224 #"Finally, notice that inception_v3 requires the input size to be (299,299), whereas all of the other models expect (224,224)."
        model = model.to(device)
        
    return model, input_size



def readData(target_type = 'breeds'):
    breed_to_species = {0:0, 1:1, 2:1, 3:1, 4:1, 5:0, 6:0, 7:0, 8:1, 9:0, 10:1,
                    11:0, 12:1, 13:1, 14:1, 15:1, 16:1, 17:1, 18:1, 19:1, 20:0,
                    21:1, 22:1, 23:0, 24:1, 25:1, 26:0, 27:0, 28:1, 29:1, 30:1,
                    31:1, 32:0, 33:0, 34:1, 35:1, 36:1
                   }
    
    transform = T.Compose([T.Resize(image_size),
                           T.CenterCrop(image_size),
                           T.ToTensor(),
                           T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                           T.RandomHorizontalFlip(p=0.5),
                           #T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
                           #T.RandomHorizontalFlip(),
                           #T.RandomRotation(degrees=60),
                           #T.RandomResizedCrop(size=image_size)
                          ])
    
    if target_type == 'species':
        target_function = (lambda x: breed_to_species.get(x))
    elif target_type == 'breeds':
        target_function = None
    else:
        raise Exception("Wrong target type")
        
    trainval = OxfordIIITPet(path,
                             target_types= "category",
                             transform=transform,
                             split = 'trainval',
                             target_transform = target_function
                            )
    test = OxfordIIITPet(path,
                         target_types= "category",
                         transform=transform,
                         split = 'test',
                         target_transform = target_function
                        )
                         
    #train, val = splitData(trainval, 0.95, 0.05)
    return trainval, test #train, val, test
    
def splitData(dataset,nrTrain,nrVal):
    len1 = int(nrTrain*len(dataset))
    len2 = int(len(dataset) - len1)
    train, val = torch.utils.data.random_split(dataset, [len1,len2])
    return train, val

def show_example(img, label):
    print('Label: ', train_dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))




In [4]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                pass#scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [5]:
def score(model, dataloader):
    running_corrects = 0
    model.eval()
    for inputs, labels in tqdm(dataloader):
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    acc = running_corrects.double() / len(dataloader.dataset)
    print('Acc: {:4f}'.format(acc))

## Model evaluation

In [6]:
train, val    = readData(
                            #target_type = 'species'
                            target_type = 'breeds'
                            )

In [7]:
#train, _ = splitData(train, 0.20, 0.8)
#val, _ = splitData(val, 0.20,0.8)

In [8]:
model,input_size = initialize_model(model_name = "resnet",
                                    num_classes = 37,
                                    feature_extracting = True)

Using cache found in /home/da150x_47/.cache/torch/hub/pytorch_vision_v0.10.0


In [9]:
dataloaders = {label: torch.utils.data.DataLoader(data, batch_size=batch_size,
                                         shuffle=True, num_workers=4)
          for label, data in zip(['train', 'val'],[train, val])}

test_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size,
                                         shuffle=False, num_workers=4)
epochs = 5
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(),
                       lr=1e-7, 
                       weight_decay=1e-5,
                       betas=(0.9, 0.999),
                      )



#scheduler = lr_scheduler.StepLR(optimizer, step_size=3,gamma=0.1)

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                max_lr=1e-3,
                                                steps_per_epoch=len(dataloaders['train']), epochs=epochs)


best_model, val_acc_history = train_model(model = model,
                                          num_epochs = epochs,
                                          dataloaders = dataloaders,
                                          criterion = criterion,
                                          optimizer = optimizer,
                                          scheduler = scheduler
                                        )

Epoch 0/4
----------
train Loss: 3.5222 Acc: 0.0712
val Loss: 2.9231 Acc: 0.3129
Epoch 1/4
----------
train Loss: 2.1224 Acc: 0.6011
val Loss: 1.4251 Acc: 0.7779
Epoch 2/4
----------
train Loss: 1.1288 Acc: 0.8370
val Loss: 0.9773 Acc: 0.8427
Epoch 3/4
----------
train Loss: 0.8451 Acc: 0.8780
val Loss: 0.8597 Acc: 0.8555
Epoch 4/4
----------
train Loss: 0.7706 Acc: 0.9000
val Loss: 0.8421 Acc: 0.8545
Training complete in 13m 33s
Best val Acc: 0.855546


In [10]:
score(best_model, test_dataloader)

  0%|          | 0/29 [00:00<?, ?it/s]

Acc: 0.856909


In [13]:
torch.save(model.state_dict(), 'tensor')

In [14]:
set_parameter_requires_grad(best_model, False)

    
optimizer = optim.Adam(model.parameters(),
                       lr=1e-6, 
                       weight_decay=1e-5,
                       betas=(0.9, 0.999),
                      )

finetune_epochs = 8
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                max_lr=1e-4,
                                                steps_per_epoch=len(dataloaders['train']),
                                                epochs=finetune_epochs)

best_model, val_acc_history = train_model(model = best_model,
                                          num_epochs = finetune_epochs,
                                          dataloaders = dataloaders,
                                          criterion = criterion,
                                          optimizer = optimizer,
                                          scheduler = scheduler
                                        )

Epoch 0/7
----------
train Loss: 0.7091 Acc: 0.9035
val Loss: 0.6186 Acc: 0.8798
Epoch 1/7
----------
train Loss: 0.4484 Acc: 0.9299
val Loss: 0.4527 Acc: 0.8972
Epoch 2/7
----------
train Loss: 0.2436 Acc: 0.9652
val Loss: 0.3963 Acc: 0.9011
Epoch 3/7
----------
train Loss: 0.1375 Acc: 0.9891
val Loss: 0.3678 Acc: 0.9024
Epoch 4/7
----------
train Loss: 0.0889 Acc: 0.9932
val Loss: 0.3460 Acc: 0.9062
Epoch 5/7
----------
train Loss: 0.0663 Acc: 0.9978
val Loss: 0.3386 Acc: 0.9111
Epoch 6/7
----------
train Loss: 0.0550 Acc: 0.9995
val Loss: 0.3365 Acc: 0.9081
Epoch 7/7
----------
train Loss: 0.0517 Acc: 1.0000
val Loss: 0.3341 Acc: 0.9087
Training complete in 34m 50s
Best val Acc: 0.911147


In [15]:
score(best_model, test_dataloader)

  0%|          | 0/29 [00:00<?, ?it/s]

Acc: 0.910330
